# BE Smart Cities
The purpose of this project is to design a house with two rooms, and model and simulate the thermal behaviour of the building. We will study this in both steady state and dynamic simulation. 

## Modeling of the rooms

The layout we chose for our rooms is the following : 

[insert rooms plan here]

## Hypothesis

For simplicity of the model, we make the following hypothesis : 
 - The thermal transfers are linear, and the radiation is approximated by a coefficient close to convection
 - Length and height of the walls are considered to be 10 times greater than the width. Therefore we consider that the thermal transfer is unidirectional, and temperature is uniform on every surface.
 - We only consider the exchanges between vertical walls, assuming the floor and ceiling are adiabatic, such as the two walls without door and windows
 - We consider the indoor surface equal to the outdoor surface
 


## Thermal model 

### Main parameters

The first step was to constitue the thermal model of the rooms, using several elements : 

- The outer walls are made out of two materials : concrete and insulation 
Their dimensions are 6 x 2.5 x 0.16 m. 

- The inner wall is made out of concrete only. 

- The windows are 0.6 x 0.8 x 0.02 m. Wall A is only glass (6 x 2.5 x 0.02 m). 

- The doors are 2.20 x 0.8 x 0.05 m.


In [1]:
import numpy as np
import pandas as pd


L = 6
wall_height = 2.5
SA = L*wall_height       # m² surface area of the glass wall (wall A)
Swindow = 0.6*0.8 # For 1 window
Sdoor= 2.2*0.8 # For 1 door
SB_c = L*wall_height-Swindow
SC_c = L*wall_height-Swindow-Sdoor
SD_c = L* wall_height
SE_c = L*wall_height-Sdoor
SF_c = L* wall_height
SG_c = L*wall_height
S_c = S_i = SB_c + SC_c + SD_c + SE_c + SF_c + SG_c
S_g= SA + 2*Swindow
S_d = 2*Sdoor
width_wall=0.16
width_insulation=0.04
width_window=0.02
width_door=0.05


ModuleNotFoundError: No module named 'numpy'

We filled in the thermo-physical properties of air, and of the 4 materials listed before.

In [ ]:
air = {'Density': 1.2,                      # kg/m³
       'Specific heat': 1000}               # J/(kg·K)
pd.DataFrame(air, index=['Air'])


concrete = {'Conductivity': 1.046,          # W/(m·K)
            'Density': 2300.0,              # kg/m³
            'Specific heat': 657,           # J/(kg⋅K)
            'Width': 0.16,                   # m
            'Surface': S_c}            # m²


insulation = {'Conductivity': 0.0325,        # W/(m·K)
              'Density': 128.0,              # kg/m³
              'Specific heat': 1093,        # J/(kg⋅K)
              'Width': 0.04,                # m
              'Surface': S_i}          # m²


glass = {'Conductivity': 1,               # W/(m·K)
         'Density': 2500,                   # kg/m³
         'Specific heat': 1210,             # J/(kg⋅K)
         'Width': 0.02,                     # m
         'Surface': S_g}                   # m²


door = {'Conductivity': 0.63,               # W/(m·K)
         'Density': 825,                   # kg/m³
         'Specific heat': 1210,             # J/(kg⋅K)
         'Width': 0.04,                     # m
         'Surface': S_d}                   # m²
        

Conventional values for the convection coefficients for indoor and outdoor convection in W/(m²⋅K) are:

In [ ]:
h = pd.DataFrame([{'in': 8., 'out': 25}], index=['h'])  # W/(m²⋅K)

Advection in the rooms 

In [2]:
# ventilation flow rate
V = L*L*wall_height                 # m³, volume of air


ACH = 1                     # 1/h, air changes per hour
Va_dot = ACH / 3600 * (V)    # m³/s, air infiltration


# ventilation & advection
Gv = air['Density'] * air['Specific heat'] * Va_dot


# P-controler gain
# Kp = 1e4            # almost perfect controller Kp -> ∞
# Kp = 1e-3           # no controller Kp -> 0
Kp = 0

NameError: name 'L' is not defined

### Electrical model of the rooms

We then drew the equivalent eletrical schema of our room and numbered each node (from 0 to 18) and each resistance (from 0 to 25). We added the sun irradiation and internal irradiation (electrical devices, users of the rooms…). 


[insert schema here]


In [ ]:
# temperature nodes
θ = ['θ0', 'θ1', 'θ2', 'θ3', 'θ4', 'θ5', 'θ6', 'θ7', 'θ8', 'θ9', 'θ10', 'θ11', 'θ12', 'θ13', 'θ14', 'θ15', 'θ16', 'θ17', 'θ18']


# flow-rate branches
q = ['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17', 'q18', 'q19', 'q20', 'q21', 'q22', 'q23', 'q24', 'q25']


# temperature nodes
nθ = 19      # number of temperature nodes
θ = [f'θ{i}' for i in range(nθ)]


# flow-rate branches
nq = 26     # number of flow branches
q = [f'q{i}' for i in range(nq)]

### Computation of matrices
The next step was to compute every value of conductance for convection and conduction and fill the conductance diagonal matrix G, the capacity diagonal matrix C, the incidence matrix A, the temperature source vector b and the heat flow source vector f

In [ ]:
G = np.array(np.hstack(
    [h['out']*SB_c,
     2 * concrete['Conductivity']*SB_c/width_wall, 2 * concrete['Conductivity']*SB_c/width_wall,
     2 * insulation['Conductivity']/width_insulation*SB_c, 2 * insulation['Conductivity']/width_insulation*SB_c,
     h['in']*SB_c,
     h['out']*SC_c,
     2 * concrete['Conductivity']/width_wall*SC_c, 2 * concrete['Conductivity']/width_wall*SC_c,
     2 * insulation['Conductivity']/width_insulation*SC_c, 2 * insulation['Conductivity']/width_insulation*SC_c,
     h['in']*SC_c,
     h['out']*SE_c,
     2 * concrete['Conductivity']/width_wall*SE_c, 2 * concrete['Conductivity']/width_wall*SE_c,
     2 * insulation['Conductivity']/width_insulation*SE_c, 2 * insulation['Conductivity']/width_insulation*SE_c,
     h['in']*SE_c,
     Gv + (h['out']+h['in'])*(SA+Sdoor) + glass['Conductivity']/width_window*SA + door['Conductivity']/width_door*Sdoor,
     Gv + (h['out']+h['in'])*(2*Swindow+Sdoor) + 2*(glass['Conductivity']/width_window*Swindow) + door['Conductivity']/width_door*Sdoor,
     Kp,
     h['in']*SG_c,
     2 * concrete['Conductivity']/width_wall*SG_c, 2 * concrete['Conductivity']/width_wall*SG_c,
     h['in']*SG_c,
     Kp]))


G_matrix=np.zeros((nq,nq))
for i in range (nq):
   G_matrix[i,i]=G[i]

pd.DataFrame(G, index=q)

